In [1]:
!pip install -r ./drum/requirements.txt

In [4]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Sequential

## Model / data parameters

In [5]:
num_classes = 10
input_shape = (28, 28, 1)

In [6]:
# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255

# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
model = Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model.summary()

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 1600)              0         
_________________________________________________________________
dropout (Dropout)            (None, 1600)              0         
_________________________________________________

## Build Model

In [7]:
batch_size = 128
epochs = 4
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

Epoch 1/4
422/422 [==============================] - 14s 32ms/step - loss: 0.7528 - accuracy: 0.7695 - val_loss: 0.0825 - val_accuracy: 0.9782
Epoch 2/4
422/422 [==============================] - 15s 35ms/step - loss: 0.1251 - accuracy: 0.9618 - val_loss: 0.0636 - val_accuracy: 0.9812
Epoch 3/4
422/422 [==============================] - 14s 34ms/step - loss: 0.0870 - accuracy: 0.9722 - val_loss: 0.0482 - val_accuracy: 0.9878
Epoch 4/4
422/422 [==============================] - 14s 34ms/step - loss: 0.0733 - accuracy: 0.9772 - val_loss: 0.0423 - val_accuracy: 0.9883


## Save Model

In [8]:
model.save("mnist.h5")

## Validate with DRUM

In [9]:
!drum --version

drum 1.4.8


In [10]:
!drum --help

usage: drum [-h] [--version]
            {score,fit,perf-test,validation,server,new,push} ...

Run user model

positional arguments:
  {score,fit,perf-test,validation,server,new,push}
                        Commands
    score               Run predictions in batch mode
    fit                 Fit your model to your data
    perf-test           Run performance tests
    validation          Run validation checks against the model
    server              serve the model via REST APIs
    new                 Create new model/env template
    push                Add your modeling code into DataRobot

optional arguments:
  -h, --help            show this help message and exit
  --version             show program's version number and exit


In [11]:
!drum perf-test -cd drum \
--input data/mnist.csv \
--target-type unstructured

DRUM performance test
Model:      /Users/tony.martin/Desktop/Jupyter/Projects/MLOps_Unstructured/MNISTforDRUM/drum
Data:       /Users/tony.martin/Desktop/Jupyter/Projects/MLOps_Unstructured/MNISTforDRUM/data/mnist.csv
# Features: 785
Preparing test data...



Running test case with timeout: 180
Running test case: 2 KB - 1 samples, 100 iterations
ProcessingRunning test case with timeout: 180
Running test case: 0.1MB - 50 samples, 50 iterations
ProcessingRunning test case with timeout: 180
Running test case: 10MB - 5062 samples, 5 iterations
ProcessingRunning test case with timeout: 180
Running test case: 50MB - 25314 samples, 1 iterations
ProcessingTest is done stopping drum server
>78Traceback (most recent call last):
  File "/Users/tony.martin/opt/anaconda3/bin/drum", line 6, in <module>
    main()
  File "/Users/tony.martin/opt/anaconda3/lib/python3.8/site-packages/datarobot_drum/drum/main.py", line 96, in main
    CMRunner(runtime).run()
  File "/Users/tony.martin/opt/anaconda3/l

In [12]:
!drum validation -cd drum \
--input data/mnist.csv \
--target-type unstructured

2021-01-18 07:54:36.681184: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-01-18 07:54:36.681398: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-01-18 07:54:36.788851: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
0,1,2,3,4,5,6,7,8,9
0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
0.0,0.0,0.0,0.0,0.0,1.0,0.

In [13]:
!drum score \
--code-dir drum \
--input data/mnist.csv \
--target-type unstructured

2021-01-18 07:54:53.643215: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-01-18 07:54:53.643402: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-01-18 07:54:53.758508: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
0,1,2,3,4,5,6,7,8,9
0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
0.0,0.0,0.0,0.0,0.0,1.0,0.

In [14]:
!drum score --code-dir drum \
--input data/mnist.csv \
--target-type multiclass \
--class-labels-file drum/classlabels.txt

2021-01-18 07:54:56.896513: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-01-18 07:54:56.896691: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-01-18 07:54:56.996658: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
      0    1    2    3    4    5    6    7    8    9
0   0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0
1   0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
2   0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
3   1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
4   0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0
5   0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0 